In [0]:
import pandas as pd

In [0]:
from google.colab import files
uploaded = files.upload()

Saving weatherHistory.csv to weatherHistory.csv


In [0]:
weather_hist = pd.read_csv('weatherHistory.csv')

In [0]:
weather_hist.head(5)

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


Найти самый холодный и самый ветреный дни за всю историю



Нас интересует день с минимальной температурой - допустим, измеренной, а не ощущаемой (-21.822222) и день с максимальной скоростью ветра (63.852600)




In [0]:
weather_hist.loc[weather_hist['Temperature (C)'] == -21.822222]

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary


Ничего хорошего не получилось; возможно, значение в таблице было округленным, а на самом деле знаков после запятой больше

In [0]:
weather_hist[weather_hist['Temperature (C)'] < -21]

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
54847,2012-02-10 07:00:00.000 +0100,Foggy,snow,-21.822222,-21.822222,0.80,3.0751,323.0,1.3685,0.0,1033.66,Foggy starting in the morning continuing until...
55490,2012-02-09 02:00:00.000 +0100,Foggy,snow,-21.111111,-21.111111,0.74,3.2200,200.0,1.9320,0.0,1034.30,Foggy until morning.
55493,2012-02-09 05:00:00.000 +0100,Foggy,snow,-21.111111,-21.111111,0.78,4.8300,180.0,2.5760,0.0,1033.30,Foggy until morning.
55494,2012-02-09 06:00:00.000 +0100,Foggy,snow,-21.111111,-21.111111,0.71,3.2200,190.0,1.9320,0.0,1032.60,Foggy until morning.


Самый холодный день - 10 февраля 2012
И уже интересно, как считали температуру, чтобы в другие 3 чуть менее холодных дня получить в дробной части 1 в периоде и ровно столько же в колонке "ощущается как" - при том, что прочие показатели не совпадают. Единственное, что приходит в голову - возможно, переводили из какой-то другой шкалы (Фаренгейт?), где числа были с меньшим количеством знаков после запятой

Самый ветреный день - 29 января 2007.

In [0]:
weather_hist[weather_hist['Wind Speed (km/h)'] > 63]

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
12181,2007-01-29 13:00:00.000 +0100,Dangerously Windy and Partly Cloudy,rain,8.944444,3.483333,0.49,63.8526,307.0,11.4471,0.0,1009.05,Mostly cloudy throughout the day and windy sta...


сколько раз шел дождь летом 2014 года

In [0]:
weather_hist['datetime'] = pd.to_datetime(weather_hist['Formatted Date'], utc=True)

In [0]:
weather_hist.dtypes

Formatted Date                           object
Summary                                  object
Precip Type                              object
Temperature (C)                         float64
Apparent Temperature (C)                float64
Humidity                                float64
Wind Speed (km/h)                       float64
Wind Bearing (degrees)                  float64
Visibility (km)                         float64
Loud Cover                              float64
Pressure (millibars)                    float64
Daily Summary                            object
datetime                    datetime64[ns, UTC]
dtype: object

In [0]:
weather_hist_2014 = weather_hist.loc[weather_hist['datetime'].dt.year == 2014]

In [0]:
weather_hist_summer2014 = weather_hist2014.loc[weather_hist2014['datetime'].dt.month > 5]
weather_hist_summer2014 = weather_hist_summer2014.loc[weather_hist_summer2014['datetime'].dt.month < 9]

In [0]:
weather_hist_summer2014_rain = weather_hist_summer2014.loc[weather_hist_summer2014['Precip Type'] == 'rain']

In [0]:
weather_hist_summer2014_rain.shape

(2205, 13)

Получается какая-то ерунда, не было летом 2014 года 2205 дней, даже если дождь шел беспрерывно

---



In [0]:
weather_hist_summer2014_rain

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary,datetime
70872,2014-08-01 00:00:00.000 +0200,Overcast,rain,20.422222,20.422222,0.90,5.6350,130.0,8.3720,0.0,1010.20,Partly cloudy throughout the day.,2014-07-31 22:00:00+00:00
70873,2014-08-01 01:00:00.000 +0200,Overcast,rain,19.983333,19.983333,0.93,6.1824,358.0,9.9820,0.0,1009.64,Partly cloudy throughout the day.,2014-07-31 23:00:00+00:00
70874,2014-08-01 02:00:00.000 +0200,Overcast,rain,19.850000,19.850000,0.93,4.8944,313.0,11.2539,0.0,1009.40,Partly cloudy throughout the day.,2014-08-01 00:00:00+00:00
70875,2014-08-01 03:00:00.000 +0200,Mostly Cloudy,rain,19.400000,19.400000,0.93,11.2700,330.0,14.9086,0.0,1009.63,Partly cloudy throughout the day.,2014-08-01 01:00:00+00:00
70876,2014-08-01 04:00:00.000 +0200,Partly Cloudy,rain,18.888889,18.888889,0.93,2.5760,128.0,11.9784,0.0,1009.89,Partly cloudy throughout the day.,2014-08-01 02:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75234,2014-06-09 21:00:00.000 +0200,Partly Cloudy,rain,22.822222,22.822222,0.53,7.6797,0.0,16.1000,0.0,1017.83,Partly cloudy starting in the morning.,2014-06-09 19:00:00+00:00
75235,2014-06-09 22:00:00.000 +0200,Partly Cloudy,rain,22.616667,22.616667,0.60,10.8514,359.0,16.1000,0.0,1018.24,Partly cloudy starting in the morning.,2014-06-09 20:00:00+00:00
75236,2014-06-09 23:00:00.000 +0200,Partly Cloudy,rain,21.166667,21.166667,0.63,3.3971,348.0,15.1823,0.0,1018.46,Partly cloudy starting in the morning.,2014-06-09 21:00:00+00:00
78189,2014-09-01 00:00:00.000 +0200,Partly Cloudy,rain,20.394444,20.394444,0.61,14.1197,160.0,16.1000,0.0,1009.19,Mostly cloudy throughout the day.,2014-08-31 22:00:00+00:00


Оказывается, каждый день проводилось несколько наблюдений. Причём посчитать уникальные дни недостаточно: к сожалению, каждый месяц нумерация начинается заново. Поэтому добавим колонку, которая будет одинаковой для измерений, сделанных в один день

In [0]:
weather_hist_summer2014_rain['date'] = weather_hist_summer2014_rain['datetime'].dt.date

In [0]:
len(weather_hist_summer2014_rain['date'].unique())

92

Летом 2014 года дождь шёл 92 дня, то есть каждый день.
Странно. Искать измерения, когда дождь не шёл, чтобы проверить этот результат, бесполезно, так как в один и тот же день дождь мог утром идти, а вечером нет. Если мы поделим длину изначальной таблицу (2205) на 24 часа в сутках (кажется, наблюдения делались раз в час), получится как раз 92. Видимо, дождь действительно шёл беспрерывно. Наверно, при таком результате надо приходить к выводу, что колонку "осадки" я перезаписала вместо того, чтобы выбрать дождливые дни, но, кажется, нет. Какое замечательное лето.

Найдём самый ветреный месяц




In [0]:
weather_hist['month'] = weather_hist['datetime'].dt.month

In [0]:
weather_hist.groupby('month').agg(['mean'])

,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars)
,mean,mean,mean,mean,mean,mean,mean,mean
month,,,,,,,,
1,0.815678,-1.938761,0.850698,11.515865,180.460655,7.835825,0.0,1006.245222
2,2.166880,-0.555908,0.813407,12.180815,188.733253,8.730536,0.0,1003.925126
3,6.914976,5.082583,0.702849,13.415825,186.632698,10.922231,0.0,1001.564999
4,12.769200,12.158287,0.641051,11.876139,186.435859,11.784991,0.0,1009.995154
5,16.874455,16.733265,0.691650,10.953186,188.143328,11.889730,0.0,1003.376970
6,20.728510,20.755415,0.686418,9.632453,202.804924,11.994000,0.0,1002.019582
7,22.965462,22.977605,0.639593,9.637124,202.937049,12.188605,0.0,1008.562854
8,22.339134,22.264600,0.635300,8.942707,195.211135,12.454307,0.0,1001.470591


Средняя скорость ветра выше всего в марте (13.4)

In [0]:
weather_hist.drop(['month'], axis=1, inplace=True)

Найдём самый влажный год

In [0]:
weather_hist['year'] = weather_hist['datetime'].dt.year

Определим, какие года представлены в таблице

In [0]:
weather_hist['year'].unique()

array([2006, 2005, 2010, 2007, 2008, 2009, 2011, 2012, 2013, 2014, 2015,
       2016])

C 2005 по 2016, без пропусков
Посмотрим, какие варианты заполнения колонки "осадки" встречаются

In [0]:
weather_hist['Precip Type'].unique()

array(['rain', 'snow', nan], dtype=object)

Видимо, влажности соответствует что угодно, кроме nan

In [0]:
most_humid_year = 0
max_humid_hours = 0
for y in range(2005, 2017):
  weather_year = weather_hist.loc[weather_hist['datetime'].dt.year == y]
  weather_year_rain = weather_year.loc[weather_year['Precip Type'] == 'rain']
  weather_year_snow = weather_year.loc[weather_year['Precip Type'] == 'snow']
  humid_hours = len(weather_year_rain['datetime'].unique()) + len(weather_year_snow['datetime'].unique()) 
  if humid_hours > max_humid_hours:
    max_humid_hours = humid_hours
    most_humid_year = y
print(y)

2016


Самый влажный год - 2016


In [0]:
weather_hist.drop(['year'], axis=1, inplace=True)

Найдём неделю с самой переменчивой погодой

In [0]:
weather_hist['week'] = weather_hist['datetime'].dt.week

Если ориентироваться на температуру, самая переменчивая погода бывает на 34 неделе года

In [0]:
weather_hist.groupby('week').agg(['std'])

,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars)
,std,std,std,std,std,std,std,std
week,,,,,,,,
1,5.008042,5.530550,0.101536,7.393594,97.107004,4.314909,0.0,246.685649
2,4.811115,5.394467,0.113069,6.531584,99.837951,4.991813,0.0,89.103028
3,4.532339,5.165160,0.107841,6.884513,91.283170,4.638049,0.0,67.561625
4,5.482606,6.102784,0.118193,7.777317,111.060259,4.480531,0.0,42.686836
5,5.251543,5.925691,0.132209,8.692668,104.658320,4.409161,0.0,122.760869
6,6.456311,7.246163,0.129590,7.094790,99.705722,4.623077,0.0,110.584848
7,4.628565,5.535001,0.127673,7.037540,99.560439,4.267868,0.0,160.344471
8,4.967552,5.908516,0.148057,7.138245,105.554739,4.156512,0.0,48.021065


## Регрессия на тексте

In [2]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('data.csv')

In [0]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [0]:
data['text_len'] = data['description'].apply(len)
data['text_len_tokens'] = data['description'].apply(lambda x: len(x.split()))
data['number_of_caps'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isupper()]))
data['number_of_nonalphanum'] = data['description'].apply(lambda x: len([ch for ch in x if not ch.isalnum()]))

### Домашнее задание 2

Попробуйте придумать признаки (как минимум 5), которые можно извлечь из текста, помимо тех 4, которые мы уже использовали. Обучите модель на них и проверьте, улучашается ли качество. Опишите признаки, которое придумали и почему считаете, что они должны показывать вероятность покупки.

Побробуйте подобрать параметры векторизатора (TfidfVectorizer или CountVectorizer, или обоих) и параметры алгоритма так, чтобы качество, которое у нас получилось (0.248) улучшилось. Опишите то, что попробовали.


Признак 1: количество чисел и последовательностей букв, обозначающих размеры одежды. Характеризует точность описания

In [0]:
import re
def size_in_text(text):
  n = len(re.findall('([0-9]+)([\.,]?)([0-9]+)?', text)) + len(re.findall('X*[SML]', text))
  return(n)


In [0]:
data['size_info'] = data['description'].apply(lambda x: size_in_text(x))

Признак 2: количесиво вопросительных и восклицательных знаков. Предполагается, что, чем их больше, тем более ненейтрально описаник

In [0]:
def question_and_exclamation_marks(text):
  question = text.count('?')
  exclamation = text.count('!')
  n = question + exclamation
  return(n)

In [0]:
data['question_and_exclamation'] = data['description'].apply(lambda x: question_and_exclamation_marks(x))

Признак 3. Доля причастий и деепричастий среди русских слов. Чем их больше, тем более книжным кажется текст

In [15]:
!pip install pymorphy2[fast]
import pymorphy2

     |████████████████████████████████| 51kB 4.2MB/s 
     |████████████████████████████████| 7.1MB 15.0MB/s 
     |████████████████████████████████| 256kB 55.4MB/s 
  Created wheel for DAWG: filename=DAWG-0.7.8-cp36-cp36m-linux_x86_64.whl size=767748 sha256=635538e5f0c6999f2305f359dc7296485551ba23a60cdfe896c3ba35c738efc6
  Stored in directory: /root/.cache/pip/wheels/d4/88/d0/4e4abc83eb8f59a71e8dbd8ba99fd5615a3af1fac1ef7f8825
Successfully built DAWG


In [0]:
morph = pymorphy2.MorphAnalyzer()
def n_participles(text, pymotphology):
    n = 0
    tokens = re.findall('[а-ёА-ЯЁ-]+', text)
    if tokens:
        n = 0
        for token in tokens:
            p = morph.parse(token)[0]
            pos = p.tag.POS
            if pos in ['PRTF', 'PRTS', 'GRND']:
                  n += 1
        p = n/len(tokens)
        return p
    else:
        return 0
     

In [0]:
data['n_participles'] = data['description'].apply(lambda x: n_participles(x, morph))

Признак 4. Наличие в тексте списков, написанных в столбик.
Предполагаем, что тексты отзывов со списками в столбик более структурированные и лучше воспринимаются. Искать будем разрывы строк после запятых и точек запятой либо числа/дeфисы в начале новой строки. Чтобы считать это списком, нужно хотя бы два однотипных указания на его наличие (списки из одного пункта не очень хорошие)

In [0]:
def contains_list(text):
    if len(re.findall('([,;])(\n)', text)) > 1:
        return 1
    elif len(re.findall('(\n)([0-9-])', text)) > 1:
        return 1
    else:
        return 0

In [0]:
data['list_in_text'] = data['description'].apply(lambda x: contains_list(x))

Признак 5. Наличие адреса в тексте. Предполагаем, что люди предпочитают договариваться о покупке, зная, куда конкретно за ней надо будет ехать

In [0]:
def adress_mentioned(text):
     adress_mentioned = 0
     if re.search('yл.?|улица|пр-т|проспект|бульвар|б-р', text):
         address_mentioned = 1
     return adress_mentioned

In [0]:
data['adress_in_text'] = data['description'].apply(lambda x: adress_mentioned(x))

Признак 6. Наличие фразы "в любое время/в любой день" (звонить/забирать/...) - такие

предложения подходят большему количеству покупателей

In [0]:
def any_time(text):
    if re.search('в любое время|в любой день', text):
        return 1
    else:
        return 0

In [0]:
data['any_time'] = data['description'].apply(lambda x: any_time(x))

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = data[['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'size_info', 'question_and_exclamation', 'n_participles', 'list_in_text', 'adress_in_text', 'any_time']].values
y = data.deal_probability.values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
from sklearn.linear_model import Ridge

In [45]:
regressor = Ridge(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.26320310910710787

От добавления метрик результат чуть-чуть ухудшился (с 4-й цифры после запятой).
Добавим векторайзер

In [0]:
from scipy.sparse import hstack

In [0]:
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2))
tfidf = TfidfVectorizer(max_features=1000, min_df=5,analyzer='char', ngram_range=(3,3), max_df=0.4)

X_cv = cv.fit_transform(data.description)
X_tfidf = tfidf.fit_transform(data.description)

X_with_CV = hstack([X_cv])
X_with_tfidf = hstack([X_tfidf])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_with_CV, y, random_state=42)

In [0]:
from sklearn.linear_model import SGDRegressor

In [53]:
regressor = SGDRegressor(max_iter=50, alpha=0.00001)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.25251747720006607

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_with_tfidf, y, random_state=42)

In [55]:
regressor = SGDRegressor(max_iter=50, alpha=0.00001)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2539645815677768

Пока что чуть-чуть лучше получается, если использовать tfidf. Соединим его с ранее использованными

In [0]:
Xtfidfcombo = hstack([X, X_with_tfidf])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(Xtfidfcombo, y, random_state=42)

In [61]:
regressor = SGDRegressor(max_iter=50, alpha=0.00001)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

48251853569864.23